In [1]:
from functions import *
from random import sample
from htmldate import find_date
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
import json
import os
from langid.langid import LanguageIdentifier, model
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib

In [2]:
columns = ["page_id","page_url","image_url","date","language","topleveldomain","category","text"]

In [3]:
# Load Date Dictionary
dates_ref = dict()

for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
    photo_folder = os.path.join(base_path, photo)
    with open(os.path.join(photo_folder,"dates.txt"),'r') as f:
        x = f.readlines()
    dates_ref.update({d.split('|')[0]:d.split('|')[-1].replace('\n','') for d in x if d.split('|')[-1].replace('\n','') != "na" and "ERROR" not in d.split('|')[-1].replace('\n','')})

# Load Language Dictionary
d_ = []
for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
    photo_folder = os.path.join(base_path, photo)
    
    with open(os.path.join(photo_folder,'languages-'+photo+".json"),'r') as f:
        lang = json.load(f)
    
    languages = []
    
    for iterkey,items1 in lang.items():
        
        for id_, lan_items in items1.items():
            
            if id_ in dates_ref.keys():
                date = dates_ref[id_]
                language = lan_items[0]
                
                d_.append([photo,language,date[0:4],1])
                
# Load Text Dictionary
dt = dict()

for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
    photo_folder = os.path.join(base_path, photo)
    num_iterations = [fol for fol in os.listdir(photo_folder) if os.path.isdir(os.path.join(photo_folder,fol)) and "source" not in fol and "context" not in fol]
    num_iterations = len(num_iterations)

    start_iter = 1
    range_iter = [str(i) for i in list(range(1,num_iterations + 1))]

    folder_base = os.path.join(base_path,photo,photo)

    for iteration in range_iter:
        fn = os.path.join(folder_base + "_" +str(iteration),"txt", "parsed_text.json")
        with open(fn) as fp:
            pages = json.load(fp)
            
        for identifier,sentences in pages.items():
            
            sentences = [s.replace("\n","").lower() for s in sentences]
            sentences = [re.sub(' +', ' ', s) for s in sentences]

            url = identifier.split('html_')[-1]
            id_ = identifier.split('/html/')[1].split('.html_')[0]
            if url in dates_ref.keys():
                date = dates_ref[url]
            else:
                date = "na"
            
            language = langid.classify(" ".join(sentences))[0]
            if language not in dt.keys():
                dt.update({language:dict()})
            
            dt[language].update({identifier:dict()})
            dt[language][identifier].update({"url":url,"identifier":id_,"date":date,"sentences":sentences})
            
            
# Define Category Function

NameError: name 'base_path' is not defined

In [7]:
# Number of Webpages
base_path = "/media/ruben/Data Drive/react-data/protest/carlo-batch-selection"


for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
    photo_folder = os.path.join(base_path, photo)

    folders = [fol for fol in os.listdir(photo_folder) if os.path.isdir(os.path.join(photo_folder,fol)) and "source" not in fol and "context" not in fol]
    print(photo, folders)
    
    for fol in folders:
        htmls = [f for f in os.listdir(os.path.join(photo_folder,fol,"html")) if ".html" in f]


11 ['11_1', '11_2', '11_3']
12 ['12_1', '12_2', '12_3', '12_4']
13 ['13_1', '13_2', '13_3', '13_4', '13_5']
14 ['14_1', '14_2', '14_3', '14_4']
16 ['16_1', '16_2', '16_3']
17 ['17_1', '17_2']
18 ['18_1', '18_2', '18_3']
19 ['19_1', '19_2']
2 ['2_1', '2_2', '2_3']
21 ['21_1', '21_2', '21_3', '21_4', '21_5']
22 ['22_1', '22_2', '22_3']
24 ['24_1', '24_2', '24_3']
25 ['25_1', '25_2', '25_3', '25_4']
27 ['27_1', '27_2', '27_3', '27_4', '27_5']
28 ['28_1', '28_2', '28_3', '28_4']
3 ['3_1', '3_2', '3_3', '3_4', '3_5', '3_6']
4 ['4_1', '4_2', '4_3']
59 ['59_1', '59_2', '59_3']
60 ['60_1', '60_2', '60_3', '60_4']
63 ['63_1', '63_2', '63_3', '63_4']
7 ['7_1', '7_2', '7_3', '7_4', '7_5', '7_6']
8 ['8_1', '8_2', '8_3', '8_4']
9 ['9_1', '9_2', '9_3', '9_4', '9_5', '9_6']


In [3]:
# Number of Scrape-Images
dt = dict()
for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
    photo_folder = os.path.join(base_path, photo)
    dt.update({photo:0})

    folders = [fol for fol in os.listdir(photo_folder) if os.path.isdir(os.path.join(photo_folder,fol)) and "source" not in fol and "context" not in fol]

    for fol in folders[:-1]:
        htmls = len([f for f in os.listdir(os.path.join(photo_folder,fol,"img")) if ".txt" not in f])
        if htmls == 0:
            dt[photo] += 1
        else:
            dt[photo] += htmls
            
d2 = pd.DataFrame(dt.items(),columns = ['image','n_scrape-images'])
d2 = d2.sort_values('image',ascending=False)
d['n_scrape-images'] = d2['n_scrape-images'] + 1
d['scrape-ratio'] = 1 - round(d['n_scrape-images'] / d['n_webpages'],2)
d = d.sort_values('image',ascending=False) 

In [4]:
# Number of Text-Scraped Pages
dt = dict()
for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
        photo_folder = os.path.join(base_path, photo)
        dt.update({photo:0})
        
        folders = [fol for fol in os.listdir(photo_folder) if os.path.isdir(os.path.join(photo_folder,fol)) and "source" not in fol and "context" not in fol]
        
        for fol in folders:
            text_jsons = [f for f in os.listdir(os.path.join(photo_folder,fol,"txt")) if ".json" in f]
            
            for js in text_jsons:
                fn = os.path.join(photo_folder,fol,"txt",js)
                with open(fn,'r') as fp:
                    js = json.load(fp)
                    list_sentences = [v for k,v in js.items()]
                    list_sentences = [len(item) for sublist in list_sentences for item in sublist]
                    dt[photo] += sum(list_sentences)
                    
d3 = pd.DataFrame(dt.items(),columns = ['image','n_scraped-text-characters'])
d3 = d3.sort_values('image',ascending=False)
d['n_scraped-text-characters'] = d3['n_scraped-text-characters']
d['char-page-ratio'] = round(d['n_scraped-text-characters'] / d['n_webpages'],2)

In [5]:
# Number of Context-Images

dt = dict()
for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
        photo_folder = os.path.join(base_path, photo)
        dt.update({photo:0})
        
        cimgs = [f for f in os.listdir(os.path.join(photo_folder,"context_images")) if ".txt" not in f]
        dt[photo] += len(cimgs)
        
d4 = pd.DataFrame(dt.items(),columns = ['image','n_context-images'])
d4 = d4.sort_values('image',ascending=False)
d['n_context-images'] = d4['n_context-images']

FileNotFoundError: [Errno 2] No such file or directory: '/media/ruben/Data Drive/react-data/protest/carlo-batch-selection/11/context_images'

In [7]:
# Number of Dates

dt = dict()
for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
        photo_folder = os.path.join(base_path, photo)
        dt.update({photo:0})
        
        with open(os.path.join(base_path,photo,"dates.txt"), 'r') as f:
            dates = f.readlines()
            dates = [d for d in dates if d.split('|')[1].replace('\n','') != "na" and "ERROR" not in d.split('|')[1]]
            dt[photo] += len(dates)
d5 = pd.DataFrame(dt.items(),columns = ['image','n_dates'])
d5 = d5.sort_values('image',ascending=False)
d['n_dates'] = d5['n_dates']

In [13]:
d['image'] = d['image'].astype(int)

In [16]:
d = d.sort_values('image',ascending=True)

In [17]:
# Export

d.to_csv('/media/ruben/FEF44259F44213F5/Users/Ruben/Documents/GitHub/ReACT_GCV/data/images_tables_article_carlo/descriptive-full.csv',index=False)

In [174]:
# Languages

dt = pd.DataFrame()

for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
    photo_folder = os.path.join(base_path, photo)
    
    with open(os.path.join(photo_folder,'languages-'+photo+".json"),'r') as f:
        lang = json.load(f)
        lang = lang['1']
        lang = dict(Counter([v[0] for k,v in lang.items()]))
        lang = pd.DataFrame(list(lang.items()))
        lang['photo'] = photo
    dt = dt.append(lang)
dt.columns = ['lang','count','photo']
d6 = dt.pivot(index="photo",columns="lang",values="count").reset_index().fillna(0)